# Worksheet 08

Name: Jian Xie  
UID: U75516303 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.8481711115241515, 4.75207816011456, 6.130432949431659, 4.828620857734927, 5.67882877568895, 5.047158100373488, 4.128431913516526, 5.0774770328695356, 3.0582290176348197, 6.120497832091548]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[8.863690497694703, 8.277298148765576, 7.993809312518509, 9.633524996618329, 7.332777514589342, 7.916592961252947, 7.607010408772074, 7.893207950661034, 7.600473832685583, 8.277378715013661]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[8.277378715013661, 7.600473832685583, 7.893207950661034, 7.607010408772074, 6.120497832091548, 3.0582290176348197, 7.916592961252947, 7.332777514589342, 5.0774770328695356, 4.128431913516526]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

1. The means of the two components: 5 and 8
2. The variances of the two components: 1 and 1
3. The proportion of points in each cluster: 0.5 and 0.5

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [10]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[3.0582290176348197, 5.0774770328695356, 4.128431913516526]
[8.277378715013661, 7.600473832685583, 7.893207950661034, 7.607010408772074, 6.120497832091548, 7.916592961252947, 7.332777514589342]
P(S_1) = 0.3,  P(S_2) = 0.7
mean_1 = 4.088045988006961,  mean_2 = 7.535419887866597
var_1 = 0.680375935994522,  var_2 = 0.4108905263886568


They are close to the true values.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [21]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  8.277378715013661
probability of observing that point if it came from cluster 0 =  3.4307433734357336e-09
probability of observing that point if it came from cluster 1 =  0.1901680771709036
point =  7.600473832685583
probability of observing that point if it came from cluster 0 =  9.570274127610972e-07
probability of observing that point if it came from cluster 1 =  0.9588282028070332
point =  7.893207950661034
probability of observing that point if it came from cluster 0 =  9.464164097071077e-08
probability of observing that point if it came from cluster 1 =  0.6645642332238617
point =  7.607010408772074
probability of observing that point if it came from cluster 0 =  9.106770910666443e-07
probability of observing that point if it came from cluster 1 =  0.9562952526602932
point =  6.120497832091548
probability of observing that point if it came from cluster 0 =  0.006767287833208443
probability of observing that point if it came from cluster 1 =  0.002583711783592683
point = 

The estimated probabilities are close from the truth.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [23]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.3,  P(S_2) = 0.7
mean_1 = 4.392648151749813,  mean_2 = 7.651056228941448
var_1 = 1.104721038488699,  var_2 = 0.26748334845061483


The values are close to the original ones except var_2.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [24]:
from scipy.stats import norm

# Updated probabilities
prob_s0_x_updated = [] # P(S_0 | X_i)
prob_s1_x_updated = [] # P(S_1 | X_i)

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x_updated = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x_updated.append( pdf_i[0] * prob_c[0] / prob_x_updated )
    prob_s1_x_updated.append( pdf_i[1] * prob_c[1] / prob_x_updated )

# Compare the updated probabilities with the original ones
for i in range(len(data)):
    print("Data point: ", data[i])
    print("Original P(S_0 | X_i): ", prob_s0_x[i])
    print("Updated P(S_0 | X_i): ", prob_s0_x_updated[i])
    print("Original P(S_1 | X_i): ", prob_s1_x[i])
    print("Updated P(S_1 | X_i): ", prob_s1_x_updated[i])
    print()

Data point:  8.277378715013661
Original P(S_0 | X_i):  7.73167925511447e-09
Updated P(S_0 | X_i):  0.0042099553610918175
Original P(S_1 | X_i):  0.9999999922683207
Updated P(S_1 | X_i):  0.9957900446389082

Data point:  7.600473832685583
Original P(S_0 | X_i):  4.277663389728769e-07
Updated P(S_0 | X_i):  0.001980059731562256
Original P(S_1 | X_i):  0.999999572233661
Updated P(S_1 | X_i):  0.9980199402684377

Data point:  7.893207950661034
Original P(S_0 | X_i):  6.103352959703415e-08
Updated P(S_0 | X_i):  0.0013114606637394523
Original P(S_1 | X_i):  0.9999999389664703
Updated P(S_1 | X_i):  0.9986885393362606

Data point:  7.607010408772074
Original P(S_0 | X_i):  4.0812711504314273e-07
Updated P(S_0 | X_i):  0.0019379830149911027
Original P(S_1 | X_i):  0.9999995918728849
Updated P(S_1 | X_i):  0.998062016985009

Data point:  6.120497832091548
Original P(S_0 | X_i):  0.5288617524327288
Updated P(S_0 | X_i):  0.9999980017454418
Original P(S_1 | X_i):  0.4711382475672712
Updated P(S_

The updated `P(S_j | X_i)` are close from the truth.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [8]:
hard_assignment = []
for i in range(len(data)):
    if prob_s0_x_updated[i] > prob_s1_x_updated[i]:
        hard_assignment.append(0)
    else:
        hard_assignment.append(1)
print(hard_assignment)

[1, 1, 1, 0, 0, 1, 1, 1, 0, 1]
